<a href="https://colab.research.google.com/github/liuezhong/homework/blob/master/homework_targetMean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext Cython

**以下是老师的源代码，耗时：0.009485960006713867**



In [ ]:
%%cython

import numpy as np
cimport numpy as np

cpdef target_mean_v3(data, y_name, x_name):
    cdef long nrow = data.shape[0]
    cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrow), dtype=np.float64)
    cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float64)
    cdef np.ndarray[double] x = np.asfortranarray(data[x_name], dtype=np.float64)

    target_mean_v3_impl(result, y, x, nrow)
    return result

cdef void target_mean_v3_impl(double[:] result, double[:] y, double[:] x, const long nrow):
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()

    cdef long i
    for i in range(nrow):
        if x[i] not in value_dict.keys():
            value_dict[x[i]] = y[i]
            count_dict[x[i]] = 1
        else:
            value_dict[x[i]] += y[i]
            count_dict[x[i]] += 1

    i=0
    for i in range(nrow):
        result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)


In [ ]:
import time
import pandas as pd

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

start = time.time()
result = target_mean_v3(data, 'y', 'x')
end = time.time()
print (end - start)

0.009485960006713867


**以下是我优化后的代码，所做的改动：**
1.   将x的类型由double改为int；
2.   用unordered_map替代dict；
3.   用for i from 0 <= i < nrow by 1替代rangge； 

耗时：0.0017247200012207031 




In [ ]:

%%cython

import numpy as np
cimport numpy as np
from libcpp.unordered_map cimport unordered_map


cpdef target_mean_v4(data, y_name, x_name):
    cdef long nrow = data.shape[0]
    cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrow), dtype=np.float64)
    cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float64)
    cdef np.ndarray[int] x = np.asfortranarray(data[x_name], dtype=np.int32)

    target_mean_v4_impl(result, y, x, nrow)
    return result

cdef void target_mean_v4_impl(double[:] result, double[:] y, int[:] x, const long nrow):
    cdef unordered_map[int, double] value_dict
    cdef unordered_map[int, int] count_dict

    cdef long i
    for i from 0 <= i < nrow by 1:
      if value_dict.find(x[i]) != value_dict.end():
          value_dict[x[i]] = y[i]
          count_dict[x[i]] = 1
      else:
          value_dict[x[i]] += y[i]
          count_dict[x[i]] += 1

    i=0
    for i from 0 <= i < nrow by 1:
        result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)



In [ ]:
import time
import pandas as pd

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

start = time.time()
result = target_mean_v4(data, 'y', 'x')
end = time.time()
print (end - start)

0.0017247200012207031
